In [2]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

D:\anaconda\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
df_old = pd.read_csv('history_auction.csv')
df_old

,auction,date_close_auction,name,year,material,quality,last_bet
0,https://www.wolmar.ru/auction/1700,29.12.2021,1 рубль. Копия,1807.0,"Ag 21,6 гр",UNC,2 084
1,https://www.wolmar.ru/auction/1700,29.12.2021,1 рубль. С подвеса,1812.0,Ag,VF,1 087
2,https://www.wolmar.ru/auction/1700,29.12.2021,"1 рубль Петров - 1,5 рубля.",1841.0,Ag,XF-,5 223
3,https://www.wolmar.ru/auction/1700,29.12.2021,"1 рубль. Отверстие Петров - 1,5 рубля.",1844.0,Ag,F,914
4,https://www.wolmar.ru/auction/1700,29.12.2021,1 рубль,1897.0,Ag,VF/XF,2 738
...,...,...,...,...,...,...,...
1228501,https://www.wolmar.ru/auction/1936,18.01.2024,Винтовки и карабины. Энциклопедия. Хартинк А.,NaN,Бумага,ХХ,638
1228502,https://www.wolmar.ru/auction/1936,18.01.2024,В Авангарде. Работы художников Гознака 1920-х ...,NaN,Бумага,ХХ,306
1228503,https://www.wolmar.ru/auction/1936,18.01.2024,Аукционный каталог фирмы Кюнкер №157. Русские ...,NaN,Бумага,ХХ,898
1228504,https://www.wolmar.ru/auction/1936,18.01.2024,Аукционный каталог фирмы Кюнкер №164. Редкие м...,NaN,Бумага,ХХ,1 379


In [5]:
auction = []
close_auction = [] 
name_lot = []
year_lot = []
material_lot = []
quality_lot = []
last_bet = []


for number_auction in range(1937,1940):
    req = requests.get("https://www.wolmar.ru/auction/"+str(number_auction))
    soup = BeautifulSoup(req.text, "html.parser")

    if re.search(r'(VIP)', soup.find('h1').text) is not None or re.search(r'(Standart)', soup.find('h1').text) is not None:
        date_close = soup.find('h1').find('span').text[8:18]
        print(date_close)
        
        for i in soup.find('ul').findAll('li')[::-1]:
            last_page = i.text
            break

        for page in range(1,int(last_page)+1):
            page_req = requests.get("https://www.wolmar.ru/auction/"+str(number_auction)+"?page="+str(page))
            page_soup = BeautifulSoup(page_req.text, "html.parser")
            
            for i in page_soup.findAll('table')[4].findAll('tr')[3:]:
                if i.findAll('td')[1].find('a') is not None:
                    auction.append("https://www.wolmar.ru/auction/"+str(number_auction))
                    close_auction.append(date_close)  
                    name_lot.append(i.findAll('td')[1].find('a').text)
                    year_lot.append(i.findAll('td')[2].text)
                    material_lot.append(i.findAll('td')[4].text)
                    quality_lot.append(i.findAll('td')[5].find('nobr').text)
                    last_bet.append(i.findAll('td')[8].text)

24.01.2024
25.01.2024


In [6]:
df = pd.DataFrame({'auction': auction,
                   'date_close_auction':close_auction,
                   'name':name_lot,
                   'year': year_lot,
                   'material':material_lot,
                   'quality': quality_lot,
                   'last_bet':last_bet})

In [12]:
df_new = pd.concat([df_old,df])

In [13]:
df_new.to_csv('history_auction.csv',
          index=False)